# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into Deep Neural Networks for NLP!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of word2vec such as vector size and window size

## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec Model. Then, you'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what your data looks like. 

In [15]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True) 
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data--`headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [24]:
df['combined_text'] = df.headline + ' ' + df.short_description
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [25]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [26]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count`. 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [28]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55562742, 67352790)

Great! you now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which is stored inside of `model.wv`. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [29]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [35]:
wv.most_similar('Texas')

[('Pennsylvania', 0.8308802843093872),
 ('Ohio', 0.815158486366272),
 ('Louisiana', 0.8137040138244629),
 ('Illinois', 0.8097048401832581),
 ('Arkansas', 0.8055933713912964),
 ('Utah', 0.7969947457313538),
 ('Maryland', 0.7937885522842407),
 ('Connecticut', 0.7896758317947388),
 ('Oregon', 0.7894039154052734),
 ('Arizona', 0.7865270376205444)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [54]:
wv.most_similar(negative=['Texas'])

[('ceases', 0.4249803423881531),
 ('prospectively', 0.422757089138031),
 ('communicates', 0.41477566957473755),
 ('🙊', 0.41366973519325256),
 ('IfIWonPowerball', 0.4135701358318329),
 ('overshadows', 0.4043700397014618),
 ('bean-to-bar', 0.4008544385433197),
 ('🎶Can', 0.39886629581451416),
 ('grieves', 0.3980882167816162),
 ('sparks', 0.3942430317401886)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [38]:
wv['Texas']

array([-3.3313406e-01,  4.0140817e-01, -7.4608797e-01, -2.6172438e-01,
        5.0486195e-01, -8.4246784e-01, -8.8866341e-01, -1.0108478e+00,
       -2.0707023e+00, -5.7320786e-01, -5.9378487e-01,  2.1567343e-02,
       -3.0403812e+00, -1.9110557e+00, -2.0657477e+00, -5.8051199e-01,
        1.7113134e+00,  9.2204946e-01, -1.5919549e+00,  1.1850970e-01,
       -5.0888717e-01, -5.8461761e-01, -1.2898279e+00,  6.2780243e-01,
       -2.8709807e+00, -2.4883540e+00,  4.3124038e-01, -2.4906011e+00,
        2.8317569e-02,  4.7677332e-01,  3.2343459e-01, -8.8336647e-01,
        4.1995564e-01,  4.3745303e-01,  4.2974824e-01,  1.3402902e+00,
       -1.4043322e-01,  1.0380775e+00, -1.6563889e+00,  4.2807570e-01,
        1.8840581e+00, -2.5545541e-02, -9.7873402e-01, -2.5912127e+00,
       -6.9881302e-01,  5.2831214e-02, -7.0610869e-01,  2.2555454e+00,
        4.0416905e-01, -1.9529815e+00,  7.9512805e-01,  1.2489452e+00,
       -1.9877939e-01, -5.2886498e-01, -8.3890200e-02,  1.4406663e+00,
      

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [39]:
wv.vectors

array([[-1.1255563e+00,  5.8268994e-01,  1.0967422e+00, ...,
        -1.1339204e-01, -1.4031552e+00,  4.2111605e-01],
       [ 4.8233349e-02,  1.3679949e+00,  1.7689906e+00, ...,
        -2.8695462e+00, -2.8340833e+00, -3.7967488e-01],
       [ 8.4575856e-01,  1.0607631e+00,  8.1978464e-01, ...,
        -2.0094640e+00, -2.1201472e+00, -2.1659516e-01],
       ...,
       [ 5.1437174e-03, -2.9052977e-02, -1.5563360e-01, ...,
         4.9952801e-02,  2.7084926e-02, -1.0551569e-01],
       [-4.7372337e-03,  3.2667790e-02,  3.2849988e-05, ...,
        -8.6145617e-02, -7.7901199e-02, -1.6527943e-02],
       [ 4.0109679e-02, -7.2684549e-02, -1.8981546e-01, ...,
         2.2707185e-02, -8.5188545e-02, -1.2354892e-01]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `most_similar` function and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [53]:
wv.most_similar(positive=['king','woman'],negative=['man'])

[('princess', 0.6117678284645081),
 ('jewel', 0.6028552651405334),
 ('title', 0.5983272790908813),
 ('queen', 0.5654557347297668),
 ('tiger', 0.5594695806503296),
 ('fan', 0.5418410301208496),
 ('spectacle', 0.5411934852600098),
 ('goddess', 0.5388637185096741),
 ('birthplace', 0.538661777973175),
 ('trappings', 0.5335830450057983)]

As you can see from the output above, your model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As you can see from the word in first place, 'reminiscent', your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to created vectorized word embeddings!